# 0.0 Data preparation

# Preparation

Imports libraries

Loads data

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib inline

print("0.0 data preparation")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.transforms as tr
import seaborn as sns
import re
import json
import datetime
import random
import os

from random import randint
from ipywidgets import FloatProgress,IntProgress,IntText,Text,interact,interactive,IntSlider,FloatSlider
from IPython.display import display
from itertools import chain
from scipy.stats import ttest_ind

In [ ]:
pd.__version__

In [ ]:
np.__version__

### common variables

In [ ]:
# questions & columns
indexColumn = 'index'
emailQuestion = 'Email'
timestampQuestion = 'Timestamp'
idQuestion = 'ID'
nameQuestion = 'Name'
gameQuestion = 'Which game have you just played?'
remarksQuestion = '12. Write here your remarks about the game: which feature was missing, what failed or glitched, what was great:'
questionCount = 12
firstLikertQuestionIndex = 0
lastLikertQuestionIndex = 11
firstSUSQuestionIndex = 0
lastSUSQuestionIndex = 10
questionArrayInt = [i+1 for i in range(questionCount)]
questionArrayStr = sorted(["Q" + "{0:0=2d}".format(i) for i in questionArrayInt])
#questionArrayStr

In [ ]:
def tryCreateFolder(path, displayMessage = False):
    try:  
        os.mkdir(path)
    except OSError:  
        if displayMessage:
            print ("Creation of the directory %s failed" % path)
    else:  
        if displayMessage:
            print ("Successfully created the directory %s " % path)

In [ ]:
# define the name of the directory to be created
graphsSavePathStem = "../Graphs"
tryCreateFolder(graphsSavePathStem)

In [ ]:
suffixSUSNormalized = "SUSNormalized"

In [ ]:
dataFolderPath = "../Data/"
unanonymizedDataFolderPath = dataFolderPath + "UnanonymizedData/"
csvSuffix = '.csv'
rawSuffix = "_event_raw"
rawAnonymizedSuffix = "_event_raw_unanonymized"
dataStem = "_data"
SUSNormalizedSuffix = "_SUSNormalized"

# Miranda House data
dateStemMirandaHouse                       = "2019-06-03"
unanonymizedDataFilesNamesStemMirandaHouse = dateStemMirandaHouse + rawAnonymizedSuffix
dataFilesNamesStemMirandaHouse             = dateStemMirandaHouse + rawSuffix
unanonymizedRaw20190603PathMirandaHouse    = unanonymizedDataFolderPath + unanonymizedDataFilesNamesStemMirandaHouse           + csvSuffix
raw20190603PathMirandaHouse                = dataFolderPath             + dataFilesNamesStemMirandaHouse                       + csvSuffix
data20190603Path                           = dataFolderPath             + dateStemMirandaHouse + dataStem                      + csvSuffix
data20190603SUSNormalizedPath              = dataFolderPath             + dateStemMirandaHouse + dataStem + SUSNormalizedSuffix + csvSuffix

# Cité des Sciences data
dateStemCiteDesSciences                       = "2019-07-03"
unanonymizedDataFilesNamesStemCiteDesSciences = dateStemCiteDesSciences + rawAnonymizedSuffix
dataFilesNamesStemCiteDesSciences             = dateStemCiteDesSciences + rawSuffix
unanonymizedRaw20190703PathCiteDesSciences    = unanonymizedDataFolderPath + unanonymizedDataFilesNamesStemCiteDesSciences + csvSuffix
raw20190703PathCiteDesSciences                = dataFolderPath             + dataFilesNamesStemCiteDesSciences             + csvSuffix
data20190703Path                              = dataFolderPath             + dateStemCiteDesSciences + dataStem            + csvSuffix
data20190703SUSNormalizedPath                 = dataFolderPath             + dateStemCiteDesSciences + dataStem + SUSNormalizedSuffix + csvSuffix

# Kigali data
dateStemKigali                       = "2019-08-28"
unanonymizedDataFilesNamesStemKigali = dateStemKigali + rawAnonymizedSuffix
dataFilesNamesStemKigali             = dateStemKigali + rawSuffix
unanonymizedRaw20190828PathKigali    = unanonymizedDataFolderPath + unanonymizedDataFilesNamesStemKigali + csvSuffix
raw20190828PathKigali                = dataFolderPath             + dataFilesNamesStemKigali             + csvSuffix
data20190828Path                     = dataFolderPath             + dateStemKigali + dataStem            + csvSuffix
data20190828SUSNormalizedPath        = dataFolderPath             + dateStemKigali + dataStem + SUSNormalizedSuffix + csvSuffix


# data anonymization

In [ ]:
# Warning: works in-place
def fixData(dataToFix):
    questionsInvolved = [gameQuestion]
    wrongSpelling = "Fungal Invasion"
    correctSpelling = "Fungal Invaders"
    for answer in dataToFix.index:
        for question in questionsInvolved:
            if dataToFix.loc[answer, question] == wrongSpelling:
                dataToFix.loc[answer, question] = correctSpelling
    return dataToFix
        

#testData = raw20190603.copy()
#fixData(testData)
#testData.loc[:, questionsInvolved]        

In [ ]:
def fixRawData(rawPath):
    raw = pd.read_csv(rawPath, index_col=indexColumn, dtype=str)
    fixData(raw)
    raw.to_csv(rawPath, encoding='utf-8')
    
#fixRawData(raw20190603PathMirandaHouse)
#fixRawData(raw20190703PathCiteDesSciences)

In [ ]:
# returns anonymized data: replaces Name by identifier and removes the Email column
# reads from the unanomizedresult to the anonymizedPath
def preprocessAnonymize(unanonymizedPath, anonymizedPath):
    
    # TODO filter out tests by our team members and organizers
    
    unanonymizedData = pd.read_csv(unanonymizedPath,dtype=str)

    unanonymizedData = unanonymizedData.drop(axis=1, columns=[emailQuestion])

    nameSet = set(unanonymizedData[nameQuestion])
    nameSeries = pd.Series(list(nameSet))
    for answerIndex in unanonymizedData.index:
        name = unanonymizedData.loc[answerIndex, nameQuestion]
        unanonymizedData.loc[answerIndex, nameQuestion] = nameSeries.index[nameSeries == name][0]
    unanonymizedData.rename(columns={nameQuestion:idQuestion}, inplace=True)

    fixData(unanonymizedData)
        
    if indexColumn in unanonymizedData.columns:
        unanonymizedData = unanonymizedData.set_index(indexColumn)
        
    print("writing to " + anonymizedPath)
    unanonymizedData.to_csv(anonymizedPath, encoding='utf-8')
    
    return unanonymizedData

In [ ]:
# Python adds an artefact index column with indices 0,..,n _by default_ when reading then writing a csv without using the argument index_col=.
# This function fixes the consequences of forgetting to use this argument.
def getRidOfArtefactFirstColumn(rawPath):
    raw = pd.read_csv(rawPath,dtype=str)
    raw = raw.drop(axis=1, columns=raw.columns[0])
    raw = raw.set_index('Timestamp')
    raw.to_csv(rawPath, encoding='utf-8')

In [ ]:
doAnonymize = False

if doAnonymize:
    raw20190603 = preprocessAnonymize(\
        unanonymizedRaw20190603PathMirandaHouse,\
        raw20190603PathMirandaHouse)
    
if doAnonymize:
    raw20190703 = preprocessAnonymize(\
        unanonymizedRaw20190703PathCiteDesSciences,\
        raw20190703PathCiteDesSciences)

if doAnonymize:
    # first merge the three separate data files
    unanonymizedPath1 = "../Data/UnanonymizedData/2019-08-28_website_Game-Questionnaire-Game1.csv"
    unanonymizedPath2 = "../Data/UnanonymizedData/2019-08-28_website_Game-Questionnaire-Game2.csv"
    unanonymizedPath3 = "../Data/UnanonymizedData/2019-08-28_website_Game-Questionnaire-Game3.csv"

    unanonymized1 = pd.read_csv(unanonymizedPath1,dtype=str)
    unanonymized2 = pd.read_csv(unanonymizedPath2,dtype=str)
    unanonymized3 = pd.read_csv(unanonymizedPath3,dtype=str)

    concatenated = pd.concat([unanonymized1, unanonymized2, unanonymized3])

    assert len(unanonymized1) + len(unanonymized2) + len(unanonymized3) == len(concatenated)

    concatenated.index = range(len(concatenated.index))
    concatenated.index.name = indexColumn
    concatenated.to_csv(unanonymizedRaw20190828PathKigali, encoding='utf-8')
    
    raw20190828 = preprocessAnonymize(\
        unanonymizedRaw20190828PathKigali,\
        raw20190828PathKigali)

# Functions

## Loading

In [ ]:
# Miranda House
try:
    raw20190603  = pd.read_csv(raw20190603PathMirandaHouse, index_col=indexColumn, dtype=str)
except:
    print("Miranda House data read failed: processing raw data...")
    raw20190603 = preprocessAnonymize(unanonymizedRaw20190603PathMirandaHouse, raw20190603PathMirandaHouse)
    raw20190603 = pd.read_csv(raw20190603PathMirandaHouse, index_col=indexColumn, dtype=str)

In [ ]:
# Cite des Sciences
try:
    raw20190703  = pd.read_csv(raw20190703PathCiteDesSciences, index_col=indexColumn, dtype=str)
except:
    print("Cite des Sciences data read failed: processing raw data...")
    raw20190703 = preprocessAnonymize(unanonymizedRaw20190703PathCiteDesSciences, raw20190703PathCiteDesSciences)
    raw20190703 = pd.read_csv(raw20190703PathCiteDesSciences, index_col=indexColumn, dtype=str)

In [ ]:
# Kigali
try:
    raw20190828  = pd.read_csv(raw20190828PathKigali, index_col=indexColumn, dtype=str)
except:
    print("Kigali data read failed: processing raw data...")
    raw20190828 = preprocessAnonymize(unanonymizedRaw20190828PathKigali, raw20190828PathKigali)
    raw20190828 = pd.read_csv(raw20190828PathKigali, index_col=indexColumn, dtype=str)

In [ ]:
assert ((raw20190603.columns == raw20190703.columns).all()), \
("column mismatch: data from different experiments must have the same columns")

## additional treatment: column name extraction
Game question
short, indexed, Likert, SUS questions

In [ ]:
firstQuestionIndex = len(raw20190603.columns)-len(questionArrayStr)

In [ ]:
shortQuestions = pd.Series(index=questionArrayStr, data=raw20190603.columns[firstQuestionIndex:])
shortQuestions

In [ ]:
indexedQuestions = pd.Series(index=range(1,13), data=raw20190603.columns[firstQuestionIndex:])
indexedQuestions

In [ ]:
indexedLikertQuestions = indexedQuestions[firstLikertQuestionIndex:lastLikertQuestionIndex]
indexedLikertQuestions

In [ ]:
positiveLikertQuestions = indexedLikertQuestions.copy()
positiveLikertQuestions = positiveLikertQuestions[positiveLikertQuestions.index % 2 == 1]
positiveLikertQuestions

In [ ]:
negativeLikertQuestions = indexedLikertQuestions.copy()
negativeLikertQuestions = negativeLikertQuestions[negativeLikertQuestions.index % 2 == 0]
negativeLikertQuestions

In [ ]:
shortLikertQuestions = shortQuestions[firstLikertQuestionIndex:lastLikertQuestionIndex]
shortLikertQuestions

In [ ]:
indexedSUSQuestions = indexedQuestions[firstSUSQuestionIndex:lastSUSQuestionIndex]
indexedSUSQuestions

In [ ]:
positiveSUSQuestions = indexedSUSQuestions.copy()
positiveSUSQuestions = positiveSUSQuestions[positiveSUSQuestions.index % 2 == 1]
positiveSUSQuestions

In [ ]:
negativeSUSQuestions = indexedSUSQuestions.copy()
negativeSUSQuestions = negativeSUSQuestions[negativeSUSQuestions.index % 2 == 0]
negativeSUSQuestions

In [ ]:
shortSUSQuestions = shortQuestions[firstSUSQuestionIndex:lastSUSQuestionIndex]
shortSUSQuestions

## 1-word question descriptions

In [ ]:
shortDescQuestions = pd.Series(index=raw20190603.columns[firstQuestionIndex:], data=[
     "frequency",				#01. I think that I would like to play this game frequently.',
     "complexity",				#02. I found the game unnecessarily complex.',
     "ease",					#03. I thought the game was easy to play.',
     "need for support",		#04. I think that I would need the support of a technical person to be able to play this game.',    
     "integration",				#05. I found the various functions in this game were well integrated.',
     "consistency",				#06. I thought there was too much inconsistency in this game.',
     "others' learning",		#07. I would imagine that most people would learn to play this game very quickly.',
     "convenience",				#08. I found the game very cumbersome to play.',
	 "confidence",				#09. I felt very confident using the game.',
	 "my learning",				#10. I needed to learn a lot of things before I could get going with this game.',
	 "recommendation",			#11. I would recommend this game to a friend.',
	 "remarks",					#12. Write here your remarks about the game: which feature was missing, what failed or glitched, what was great:',
	 ])
shortDescQuestions

In [ ]:
shortDescQuestions = pd.Series(index=raw20190603.columns[firstQuestionIndex:], data=[
     "replay appeal",			#01. I think that I would like to play this game frequently.',
     "simplicity",				#02. I found the game unnecessarily complex.',
     "ease",					#03. I thought the game was easy to play.',
     "autonomy",		        #04. I think that I would need the support of a technical person to be able to play this game.',    
     "integration",				#05. I found the various functions in this game were well integrated.',
     "consistency",				#06. I thought there was too much inconsistency in this game.',
     "learnable by others",		#07. I would imagine that most people would learn to play this game very quickly.',
     "convenience",				#08. I found the game very cumbersome to play.', maniability
	 "confidence",				#09. I felt very confident using the game.',
	 "learnable by self",	    #10. I needed to learn a lot of things before I could get going with this game.',
	 "recommendation",			#11. I would recommend this game to a friend.',
	 "remarks",					#12. Write here your remarks about the game: which feature was missing, what failed or glitched, what was great:',
	 ])
shortDescQuestions

## game names

In [ ]:
games = raw20190603[gameQuestion].unique()
games

In [ ]:
gameDrBugTitle = 'Dr Bug: Microbe Mayhem'
gameSuperbugsTitle = 'Superbugs: the Game'
gameFungalTitle = 'Fungal Invaders'

assert (set(games)==set((gameDrBugTitle, gameSuperbugsTitle, gameFungalTitle))), ("Wrong list of games")

In [ ]:
identityGameNames = pd.Series(index=[gameDrBugTitle, gameSuperbugsTitle, gameFungalTitle],
                           data=[gameDrBugTitle, gameSuperbugsTitle, gameFungalTitle])
shortGameNames = pd.Series(index=identityGameNames.index,
                           data=['Dr Bug', 'Superbugs', 'Fungal Invaders'])
shortGameNames

In [ ]:
def getShortGameTitle(longGameName):
    return shortGameNames.get(longGameName, "unknown")

## advanced treatment: data refinement
Scores are stored as strings and must be converted to integers and NaNs for missing values

In [ ]:
def getNumericalData(anonymizedData):
    numericalData = anonymizedData.copy()

    for question in indexedLikertQuestions:
        for respondent in numericalData.index:
            try:
                numericalData.loc[respondent, question] = int(numericalData.loc[respondent, question])
            except:
                numericalData.loc[respondent, question] = np.nan
    return numericalData

In [ ]:
def getNormalizedNumericalData(numericalData):
    # transforms the agreement scores (1-5 Likert scale) into a 0-4 mark with 0 = bad and 4 = great
    normalizedNumericalData = numericalData.copy()
    for respondent in normalizedNumericalData.index:
        for question in indexedLikertQuestions:
            answer = normalizedNumericalData.loc[respondent, question]
            if pd.notna(answer):
                if question in negativeLikertQuestions.values:            
                    normalizedNumericalData.loc[respondent, question] = 5-answer
                else:
                    normalizedNumericalData.loc[respondent, question] = answer-1
                    
    return normalizedNumericalData

In [ ]:
# Miranda House data

try:
    data20190603               = pd.read_csv(data20190603Path, index_col=indexColumn, dtype=str)
    data20190603SUSNormalized  = pd.read_csv(data20190603SUSNormalizedPath, index_col=indexColumn, dtype=str)
except:
    print("data20190603 read failed: processing raw data...")
    data20190603 = getNumericalData(raw20190603)
    data20190603SUSNormalized = getNormalizedNumericalData(data20190603)
    
    data20190603.to_csv(data20190603Path, encoding='utf-8')
    data20190603SUSNormalized.to_csv(data20190603SUSNormalizedPath, encoding='utf-8')
    
    data20190603               = pd.read_csv(data20190603Path, index_col=indexColumn, dtype=str)
    data20190603SUSNormalized  = pd.read_csv(data20190603SUSNormalizedPath, index_col=indexColumn, dtype=str)


In [ ]:
# Cite des Sciences data

try:
    data20190703               = pd.read_csv(data20190703Path, index_col=indexColumn, dtype=str)
    data20190703SUSNormalized  = pd.read_csv(data20190703SUSNormalizedPath, index_col=indexColumn, dtype=str)
except:
    print("data20190703 read failed: processing raw data...")
    data20190703 = getNumericalData(raw20190703)
    data20190703SUSNormalized = getNormalizedNumericalData(data20190703)
    
    data20190703.to_csv(data20190703Path, encoding='utf-8')
    data20190703SUSNormalized.to_csv(data20190703SUSNormalizedPath, encoding='utf-8')
    
    data20190703               = pd.read_csv(data20190703Path, index_col=indexColumn, dtype=str)
    data20190703SUSNormalized  = pd.read_csv(data20190703SUSNormalizedPath, index_col=indexColumn, dtype=str)

In [ ]:
# Kigali data

try:
    data20190828               = pd.read_csv(data20190828Path, index_col=indexColumn, dtype=str)
    data20190828SUSNormalized  = pd.read_csv(data20190828SUSNormalizedPath, index_col=indexColumn, dtype=str)
except:
    print("data20190828 read failed: processing raw data...")
    data20190828 = getNumericalData(raw20190828)
    data20190828SUSNormalized = getNormalizedNumericalData(data20190828)
    
    data20190828.to_csv(data20190828Path, encoding='utf-8')
    data20190828SUSNormalized.to_csv(data20190828SUSNormalizedPath, encoding='utf-8')
    
    data20190828               = pd.read_csv(data20190828Path, index_col=indexColumn, dtype=str)
    data20190828SUSNormalized  = pd.read_csv(data20190828SUSNormalizedPath, index_col=indexColumn, dtype=str)

In [ ]:
datasets = dict()

datasets["data20190603"] = data20190603
datasets["data20190603SUSNormalized"] = data20190603SUSNormalized
datasets["data20190703"] = data20190703
datasets["data20190703SUSNormalized"] = data20190703SUSNormalized
datasets["data20190828"] = data20190828
datasets["data20190828SUSNormalized"] = data20190828SUSNormalized

In [ ]:
def selectDataset(datasetName):
    #_dataName = "data20190603"
    _dataName = datasetName
    assert (_dataName in datasets), ("Not found in datasets: '" + _dataName + "'")
    inputData=datasets[_dataName]

    _dataNameSUSNormalized = _dataName + suffixSUSNormalized
    assert (_dataName in datasets), ("Not found in datasets: '" + _dataNameSUSNormalized + "'")
    inputDataSUSNormalized=datasets[_dataNameSUSNormalized]
    
    return _dataName, inputData, _dataNameSUSNormalized, inputDataSUSNormalized

In [ ]:
# digital5StepDescriptions = range(minLikertValue, maxLikertValue+1)
# mixed5StepDecriptions = ['Strongly agree', '2', '3', '4', 'Strongly disagree']
# likert5StepDescriptions = ['Strongly agree', 'Slightly agree', 'Neutral', 'Slightly disagree', 'Strongly disagree']
likert5StepDescriptions = ['Strongly agree', 'Agree', 'Neutral', 'Disagree', 'Strongly disagree']